In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2
import os

from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import optimizers
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [43]:
num_classes = 3

channels = 3 #RGB channels

image_resize = 224 #necessary resolution

num_epochs = 10
early_stop_patience = 3

steps_per_epoch_training = 10
steps_per_epoch_validation = 10

batch_size_training = 100
batch_size_validation = 100
batch_size_testing = 1

# Task 1: first layers are not learning

In [50]:
model = Sequential()

model.add(ResNet50(include_top = False, pooling = 'avg', weights = 'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')) #loading pretrained weights

model.add(Dense(num_classes, activation = 'softmax'))

model.layers[0].trainable = False # ResNet model (layer 1. not training)

ValueError: Shapes (1, 1, 256, 512) and (512, 128, 1, 1) are incompatible

In [29]:
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True) # tune model
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy']) #compile model

# Dataset description

The 600 images were downloaded from OpenImages via OIDv4 tool. I had to install a Python script which I could use from the command line. I could give the needed image numbers (600) per category. The tool automatically created a file tree which could be used by the DataFlow generator. 

In [46]:
image_size = image_resize

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# I got the preprocessing steps from here: https://medium.com/datadriveninvestor/deep-learning-using-transfer-learning-python-code-for-resnet50-8acdfb3a2d38 and here: https://www.kaggle.com/suniliitb96/tutorial-keras-transfer-learning-with-resnet50/
train_generator = data_generator.flow_from_directory( #create train data (easy to do thanked to the flow generator, only needed to create a proper file tree)
        'Dataset_nl/train',
        target_size=(image_size, image_size), 
        batch_size=batch_size_training,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory( #create validation data
        'Dataset_nl/valid',
        target_size=(image_size, image_size),
        batch_size=batch_size_validation,
        class_mode='categorical') 

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [34]:
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = early_stop_patience)
cb_checkpointer = ModelCheckpoint(filepath = 'best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [37]:
test_generator = data_generator.flow_from_directory( #create test data
    directory = 'Dataset_nl/test',
    target_size = (image_size, image_size),
    batch_size = batch_size_testing,
    class_mode = None,
    shuffle = False,
    seed = 123
)

Found 300 images belonging to 3 classes.


In [38]:
test_generator.reset()

pred = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)

predicted_class_indices = np.argmax(pred, axis = 1)

300/300 [==============================] - 7s 25ms/step


# Task 2: upper layers are learning

Now the FC layers are also learning and the pre-trained weights made it fast and memory efficient to train.

In [53]:
model = Sequential()

model.add(ResNet50(include_top = True, pooling = 'avg', weights = 'resnet50_weights_tf_dim_ordering_tf_kernels_top.h5')) #loading pretrained weights

model.add(Dense(num_classes, activation = 'softmax'))

model.layers[0].trainable = True # ResNet model (layer 1 training)

ValueError: Shapes (1, 1, 256, 512) and (512, 128, 1, 1) are incompatible

In [58]:
# I got the preprocessing steps from here: https://medium.com/datadriveninvestor/deep-learning-using-transfer-learning-python-code-for-resnet50-8acdfb3a2d38 and here: https://www.kaggle.com/suniliitb96/tutorial-keras-transfer-learning-with-resnet50/
train_generator = data_generator.flow_from_directory( #create train data (easy to do thanked to the flow generator, only needed to create a proper file tree)
        'Dataset_nl/train',
        target_size=(image_size, image_size), 
        batch_size=batch_size_training,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory( #create validation data
        'Dataset_nl/valid',
        target_size=(image_size, image_size),
        batch_size=batch_size_validation,
        class_mode='categorical') 

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [54]:
sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True) # tune model
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy']) #compile model

In [55]:
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = early_stop_patience)
cb_checkpointer = ModelCheckpoint(filepath = 'best_top.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [56]:
test_generator = data_generator.flow_from_directory( #create test data
    directory = 'Dataset_nl/test',
    target_size = (image_size, image_size),
    batch_size = batch_size_testing,
    class_mode = None,
    shuffle = False,
    seed = 123
)

Found 300 images belonging to 3 classes.


In [57]:
test_generator.reset()

pred = model.predict_generator(test_generator, steps = len(test_generator), verbose = 1)

predicted_class_indices = np.argmax(pred, axis = 1)

300/300 [==============================] - 5s 16ms/step


In [ ]:
validation_generator.reset()

pred = model.predict_generator(validation_generator, steps = len(validation_generator), verbose = 1)

predicted_class_indices = np.argmax(pred, axis = 1)